In [1]:
# Imports
import subprocess
import os
from joblib import Parallel, delayed
import pandas as pd
from sklearn.metrics import roc_curve, roc_auc_score
import matplotlib.pyplot as plt

In [2]:
dir = "1a4g_na"
abbr = "na"
receptor = "1A4G"  # prefix of the receptor file
!gunzip $dir/$abbr-decoys.sdf.gz
!gunzip $dir/$abbr-ligands.sdf.gz

gunzip: can't stat: 1a4g_na/na-decoys.sdf.gz (1a4g_na/na-decoys.sdf.gz.gz): No such file or directory
gunzip: can't stat: 1a4g_na/na-ligands.sdf.gz (1a4g_na/na-ligands.sdf.gz.gz): No such file or directory


In [3]:
# Create pdbqt files of ligands (true and decoy) using obabel
!obabel $dir/$abbr-ligands.sdf -O $dir/$abbr-ligands.pdbqt
!obabel $dir/$abbr-decoys.sdf -O $dir/$abbr-decoys.pdbqt

40 molecules converted
1200 molecules converted


In [4]:
def split_pdbqt(input_file, output_prefix):
    """
    Splits a multi-model PDBQT file into individual files,
    stripping MODEL/ENDMDL for Vina compatibility.
    """
    with open(input_file, 'r') as file:
        lines = file.readlines()

    model = []
    model_count = 0
    inside_model = False

    for line in lines:
        if line.startswith("MODEL"):
            inside_model = True
            model = []
        elif line.startswith("ENDMDL"):
            inside_model = False
            # Write model to file
            output_file = f"{output_prefix}_{model_count:03d}.pdbqt"
            with open(output_file, 'w') as out:
                out.writelines(model)
            model_count += 1
        elif inside_model:
            model.append(line)

    print(f"Saved {model_count} ligand files to {output_prefix}_*.pdbqt")


# Example usage
def split_ligands_and_decoys(dir, abbr):
    output_dir_ligands = os.path.join(dir, "split_ligands")
    output_dir_decoys = os.path.join(dir, "split_decoys")
    os.makedirs(output_dir_ligands, exist_ok=True)
    os.makedirs(output_dir_decoys, exist_ok=True)

    input_ligands = os.path.join(dir, f"{abbr}-ligands.pdbqt")
    input_decoys = os.path.join(dir, f"{abbr}-decoys.pdbqt")

    split_pdbqt(input_ligands, os.path.join(output_dir_ligands, "ligand"))
    split_pdbqt(input_decoys, os.path.join(output_dir_decoys, "decoy"))


In [5]:
split_ligands_and_decoys(dir, abbr)

Saved 40 ligand files to 1a4g_na/split_ligands/ligand_*.pdbqt
Saved 1200 ligand files to 1a4g_na/split_decoys/decoy_*.pdbqt


In [6]:
output_dir_ligands = f"{dir}/split_ligands"
os.makedirs(output_dir_ligands, exist_ok=True)

output_dir_decoys = f"{dir}/split_decoys"
os.makedirs(output_dir_decoys, exist_ok=True)

In [7]:
receptor_file = f"{dir}/{receptor}-receptor.pdbqt"
output_dir = f"{dir}/vs_results_ligands"
os.makedirs(output_dir, exist_ok=True)

ligands = sorted([f for f in os.listdir(output_dir_ligands) if f.endswith(".pdbqt")])

def dock(lig, input_path):
    try:
        lig_path = os.path.join(input_path, lig)
        out_path = os.path.join(output_dir, lig.replace(".pdbqt", "_docked.pdbqt"))
        
        if not os.path.isfile(lig_path):
            print(f"Ligand not found: {lig_path}")

        cmd = [
            "vina",
            "--receptor", receptor_file,
            "--ligand", lig_path,
            "--center_x", "15.190",
            "--center_y", "53.903",
            "--center_z", "16.917",
            "--size_x", "20.0",
            "--size_y", "20.0",
            "--size_z", "20.0",
            "--exhaustiveness", "16",
            "--seed", "12345",
            "--out", out_path
        ]

        subprocess.run(cmd, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Docked: {lig}")
        return f"success: {lig}"
    except subprocess.CalledProcessError as e:
        print(f"Docking failed: {lig}")
        return f"failed: {lig}"
    except Exception as e:
        print(f"Error for {lig}: {e}")
        return f"failed: {lig}"

# run in parallel
results = Parallel(n_jobs=os.cpu_count())(delayed(dock)(lig, output_dir_ligands) for lig in ligands)

Docked: ligand_001.pdbqt
Docked: ligand_004.pdbqt
Docked: ligand_003.pdbqt
Docked: ligand_005.pdbqt
Docked: ligand_002.pdbqt
Docked: ligand_007.pdbqt


In [ ]:
receptor_file = f"{dir}/{receptor}-receptor.pdbqt"
output_dir = f"{dir}/vs_results_decoys"
os.makedirs(output_dir, exist_ok=True)

ligands = sorted([f for f in os.listdir(output_dir_decoys) if f.endswith(".pdbqt")])

results = Parallel(n_jobs=os.cpu_count())(delayed(dock)(lig, output_dir_decoys) for lig in ligands)